# Chapter 4 Guide

#### note: headings in this notebook correspond with figures in the text. 

## 4.1

In [2]:
# An example prompt to get us started is below.
# Prompt: "Write a Python script that takes a list of numbers, queries the NumbersAPI for trivia facts, and stores the results in a pandas DataFrame. Handle errors gracefully if a request fails."
# The AI will generate something similar to the following script:
# Response:
import requests
import pandas as pd
import logging

# Set up logging to capture API results and errors
logging.basicConfig(filename='api_requests.log', level=logging.INFO)

def fetch_number_facts(numbers):
    data = []
    for num in numbers:
        try:
            response = requests.get(f"http://numbersapi.com/{num}?json")
            if response.status_code == 200:
                fact = response.json().get('text', 'No fact available')
                data.append({'Number': num, 'Fact': fact})
                logging.info(f"Successfully retrieved fact for {num}")
            else:
                data.append({'Number': num, 'Fact': f"Error: Status {response.status_code}"})
                logging.error(f"Failed to retrieve fact for {num}, Status: {response.status_code}")
        except Exception as e:
            data.append({'Number': num, 'Fact': f"Error: {str(e)}"})
            logging.error(f"Error occurred while fetching fact for {num}: {str(e)}")
    
    return pd.DataFrame(data)

# Example usage
numbers = [42, 7, 100]
df = fetch_number_facts(numbers)
display(df)


,Number,Fact
0,42,42 is the answer to the Ultimate Question of L...
1,7,"7 is the number of seconds it takes ""Superman:..."
2,100,100 is the number of runs required for a crick...


## 4.2

In [4]:
# Prompt: "Modify the script to include retry logic for API requests that fail due to a timeout. Retry up to 3 times with a 5-second delay between attempts. Also, print a message to the console each time the script begins processing a number."
# The AI will update the script to incorporate retry logic, reducing failures due to temporary network issues:
# Response:
import requests
import pandas as pd
import logging
import time

# Set up logging
logging.basicConfig(filename='api_requests.log', level=logging.INFO)

def fetch_number_facts(numbers):
    data = []
    for num in numbers:
        print(f"Processing number: {num}")  # Visible in console
        logging.info(f"Processing number: {num}")
        attempts = 0
        success = False
        while attempts < 3 and not success:
            try:
                response = requests.get(f"http://numbersapi.com/{num}?json", timeout=10)
                if response.status_code == 200:
                    fact = response.json().get('text', 'No fact available')
                    data.append({'Number': num, 'Fact': fact})
                    print(f"Success: Retrieved fact for {num}")  # Optional
                    logging.info(f"Successfully retrieved fact for {num}")
                    success = True
                else:
                    print(f"Error: Status {response.status_code} for number {num}")
                    logging.error(f"Failed to retrieve fact for {num}, Status: {response.status_code}")
                    attempts += 1
                    time.sleep(5)
            except requests.exceptions.Timeout:
                attempts += 1
                print(f"Timeout: Attempt {attempts} for number {num}")
                logging.warning(f"Timeout while fetching fact for {num}, attempt {attempts}")
                time.sleep(5)
            except Exception as e:
                print(f"Unexpected error for {num}: {str(e)}")
                logging.error(f"Unexpected error for {num}: {str(e)}")
                break
    
    return pd.DataFrame(data)

# Example usage
numbers = [42, 7, 100]
df = fetch_number_facts(numbers)
display(df)


Processing number: 42
Success: Retrieved fact for 42
Processing number: 7
Success: Retrieved fact for 7
Processing number: 100
Success: Retrieved fact for 100


,Number,Fact
0,42,42 is the number of kilometers in a marathon.
1,7,7 is the number of days in a week.
2,100,100 is the number of pounds in an American sho...


# 4.3

In [5]:
# Enhanced Prompt:
# I’ve uploaded the NumbersAPI reference file. Using that, please write a Python script that retrieves trivia facts for a list of numbers (e.g., 17, 23, 88). Use the JSON response format, and include the notfound=floor and fragment=true query parameters. Return the results as a pandas DataFrame and include basic error handling.
# Response:
import requests
import pandas as pd

def fetch_number_facts(numbers):
    data = []

    for num in numbers:
        print(f"Processing number: {num}")
        try:
            url = f"http://numbersapi.com/{num}/trivia?json=true&notfound=floor&fragment=true"
            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                json_data = response.json()
                data.append({
                    'Number': json_data.get('number', num),
                    'Fact': json_data.get('text', 'No fact available'),
                    'Found': json_data.get('found', False)
                })
            else:
                print(f"Failed to fetch data for {num}. Status code: {response.status_code}")
        except Exception as e:
            print(f"Error fetching data for {num}: {e}")
            data.append({'Number': num, 'Fact': 'Error occurred', 'Found': False})
    
    return pd.DataFrame(data)

# Example usage
numbers = [17, 23, 88]
df = fetch_number_facts(numbers)
display(df)


Processing number: 17
Processing number: 23
Processing number: 88


,Number,Fact,Found
0,17,"'the least random number' as described at MIT,...",True
1,23,the number of chromosomes normal human sex cel...,True
2,88,the approximate number of days it takes Mercur...,True


## 4.4

In [7]:
# Initial Prompt: "Fetch the user data from JSONPlaceholder and create a pandas DataFrame with name, email, and address.city"
# Response:
import requests
import pandas as pd

url = "https://jsonplaceholder.typicode.com/users"
response = requests.get(url)
users = response.json()

data = []
for user in users:
    data.append({
        "Name": user.get("name"),
        "Email": user.get("email"),
        "City": user.get("address", {}).get("city")
    })

df = pd.DataFrame(data)
display(df.head())


,Name,Email,City
0,Leanne Graham,Sincere@april.biz,Gwenborough
1,Ervin Howell,Shanna@melissa.tv,Wisokyburgh
2,Clementine Bauch,Nathan@yesenia.net,McKenziehaven
3,Patricia Lebsack,Julianne.OConner@kory.org,South Elvis
4,Chelsey Dietrich,Lucio_Hettinger@annie.ca,Roscoeview


# 4.5

In [9]:
# Prompt: "Fetch the user data from JSONPlaceholder. Create a DataFrame with the following fields: name, username, email, company.name, company.catchPhrase, address.city, and the coordinates from address.geo combined into one string column called Geo (lat, lng)."
# Response:
import requests
import pandas as pd

url = "https://jsonplaceholder.typicode.com/users"
response = requests.get(url)
users = response.json()

data = []
for user in users:
    geo = user.get("address", {}).get("geo", {})
    lat = geo.get("lat")
    lng = geo.get("lng")
    geo_string = f"({lat}, {lng})" if lat and lng else None

    data.append({
        "Name": user.get("name"),
        "Username": user.get("username"),
        "Email": user.get("email"),
        "Company": user.get("company", {}).get("name"),
        "Catch Phrase": user.get("company", {}).get("catchPhrase"),
        "City": user.get("address", {}).get("city"),
        "Geo (lat, lng)": geo_string
    })

df = pd.DataFrame(data)
display(df.head())


,Name,Username,Email,Company,Catch Phrase,City,"Geo (lat, lng)"
0,Leanne Graham,Bret,Sincere@april.biz,Romaguera-Crona,Multi-layered client-server neural-net,Gwenborough,"(-37.3159, 81.1496)"
1,Ervin Howell,Antonette,Shanna@melissa.tv,Deckow-Crist,Proactive didactic contingency,Wisokyburgh,"(-43.9509, -34.4618)"
2,Clementine Bauch,Samantha,Nathan@yesenia.net,Romaguera-Jacobson,Face to face bifurcated interface,McKenziehaven,"(-68.6102, -47.0653)"
3,Patricia Lebsack,Karianne,Julianne.OConner@kory.org,Robel-Corkery,Multi-tiered zero tolerance productivity,South Elvis,"(29.4572, -164.2990)"
4,Chelsey Dietrich,Kamren,Lucio_Hettinger@annie.ca,Keebler LLC,User-centric fault-tolerant solution,Roscoeview,"(-31.8129, 62.5342)"


# 4.6

In [10]:
# Prompt: "Fetch the user data from JSONPlaceholder. Create a DataFrame with the following fields: name, username, email, company.name, company.catchPhrase, address.city, and the coordinates from address.geo combined into one string column called Geo (lat, lng)."
# Response:
import requests
import pandas as pd

url = "https://jsonplaceholder.typicode.com/users"
response = requests.get(url)
users = response.json()

data = []
for user in users:
    geo = user.get("address", {}).get("geo", {})
    lat = geo.get("lat")
    lng = geo.get("lng")
    geo_string = f"({lat}, {lng})" if lat and lng else None

    data.append({
        "Name": user.get("name"),
        "Username": user.get("username"),
        "Email": user.get("email"),
        "Company": user.get("company", {}).get("name"),
        "Catch Phrase": user.get("company", {}).get("catchPhrase"),
        "City": user.get("address", {}).get("city"),
        "Geo (lat, lng)": geo_string
    })

df = pd.DataFrame(data)
display(df.head())


,Name,Username,Email,Company,Catch Phrase,City,"Geo (lat, lng)"
0,Leanne Graham,Bret,Sincere@april.biz,Romaguera-Crona,Multi-layered client-server neural-net,Gwenborough,"(-37.3159, 81.1496)"
1,Ervin Howell,Antonette,Shanna@melissa.tv,Deckow-Crist,Proactive didactic contingency,Wisokyburgh,"(-43.9509, -34.4618)"
2,Clementine Bauch,Samantha,Nathan@yesenia.net,Romaguera-Jacobson,Face to face bifurcated interface,McKenziehaven,"(-68.6102, -47.0653)"
3,Patricia Lebsack,Karianne,Julianne.OConner@kory.org,Robel-Corkery,Multi-tiered zero tolerance productivity,South Elvis,"(29.4572, -164.2990)"
4,Chelsey Dietrich,Kamren,Lucio_Hettinger@annie.ca,Keebler LLC,User-centric fault-tolerant solution,Roscoeview,"(-31.8129, 62.5342)"


In [11]:
# Prompt: "Write a Python script that extracts U.S. phone numbers from this block of text. ‘Call me at (212) 555-7890. My backup number is 646-123-4567. You can also reach me at 917.555.1234.’"
# Response:
import re

text = """
Call me at (212) 555-7890. My backup number is 646-123-4567. You can also reach me at 917.555.1234.
"""

# Regex to match U.S. phone numbers
phone_numbers = re.findall(r'\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}', text)
print(phone_numbers)


['(212) 555-7890', '646-123-4567', '917.555.1234']


In [12]:
# Prompt: "Modify the script to normalize the extracted phone numbers to the format +1-XXX-XXX-XXXX."
# Response:
import re

text = """
Call me at (212) 555-7890. My backup number is 646-123-4567. You can also reach me at 917.555.1234.
"""

# Extract raw phone numbers
raw_numbers = re.findall(r'\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}', text)

# Normalize each number to +1-XXX-XXX-XXXX
def normalize(number):
    digits = re.sub(r'\D', '', number)  # Remove non-digit characters
    if len(digits) == 10:
        return f"+1-{digits[0:3]}-{digits[3:6]}-{digits[6:]}"
    return number

normalized_numbers = [normalize(num) for num in raw_numbers]
print(normalized_numbers)


['+1-212-555-7890', '+1-646-123-4567', '+1-917-555-1234']


# 4.7

In [13]:
# Prompt: "Write a Python script that extracts U.S. phone numbers from text and splits each into three fields: area_code, first_3, and last_4. Return the result as a pandas DataFrame."
# Response:
import re
import pandas as pd

text = """
Call me at (212) 555-7890. My backup number is 646-123-4567. You can also reach me at 917.555.1234.
"""

# Regex with capture groups for structured output
matches = re.findall(r'\(?(\d{3})\)?[-.\s]?(\d{3})[-.\s]?(\d{4})', text)

# Convert to DataFrame
df = pd.DataFrame(matches, columns=["Area Code", "First 3", "Last 4"])
display(df)


,Area Code,First 3,Last 4
0,212,555,7890
1,646,123,4567
2,917,555,1234
